In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Embedding
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.datasets import imdb
import numpy as np

from tensorflow.keras.layers import Dense,LSTM
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [2]:
(X_train_seq, Y_train), (X_test_seq, Y_test) = imdb.load_data()

vocabulary_size = -1

for seq in X_train_seq:
    max_index = max(seq)
    if max_index > vocabulary_size:
        vocabulary_size = max_index

for seq in X_test_seq:
    max_index = max(seq)
    if max_index > vocabulary_size:
        vocabulary_size = max_index

vocabulary_size += 1

X_train = sequence.pad_sequences(X_train_seq, maxlen=100)
X_test = sequence.pad_sequences(X_test_seq, maxlen=100)

In [3]:
print(np.shape(X_train_seq[2]), np.shape(Y_train), np.shape(X_test_seq[2]), np.shape(Y_test))
print(np.shape(X_train), np.shape(X_test))

(141,) (25000,) (603,) (25000,)
(25000, 100) (25000, 100)


In [4]:
model = Sequential()
model.add(Embedding(output_dim=300, input_dim=vocabulary_size, input_length=100))

model.add(LSTM(64,return_sequences=True))  #返回所有节点的输出
# model.add(LSTM(32,return_sequences=True))  
model.add(LSTM(32,return_sequences=False))  #返回最后一个节点的输出
# model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))


model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          26576100  
_________________________________________________________________
lstm (LSTM)                  (None, 100, 64)           93440     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 26,681,989
Trainable params: 26,681,989
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.fit(X_train, Y_train, epochs=10, batch_size=128)

Epoch 1/10
196/196 [==============================] - 9s 13ms/step - loss: 0.4248 - accuracy: 0.8056
Epoch 2/10
196/196 [==============================] - 3s 13ms/step - loss: 0.2487 - accuracy: 0.9024
Epoch 3/10
196/196 [==============================] - 3s 13ms/step - loss: 0.1711 - accuracy: 0.9344
Epoch 4/10
196/196 [==============================] - 3s 13ms/step - loss: 0.1138 - accuracy: 0.9590
Epoch 5/10
196/196 [==============================] - 2s 12ms/step - loss: 0.0786 - accuracy: 0.9742
Epoch 6/10
196/196 [==============================] - 2s 12ms/step - loss: 0.0512 - accuracy: 0.9821
Epoch 7/10
196/196 [==============================] - 2s 13ms/step - loss: 0.0326 - accuracy: 0.9892
Epoch 8/10
196/196 [==============================] - 2s 13ms/step - loss: 0.0244 - accuracy: 0.9920
Epoch 9/10
196/196 [==============================] - 3s 13ms/step - loss: 0.0179 - accuracy: 0.9950
Epoch 10/10
196/196 [==============================] - 2s 12ms/step - loss: 0.0181 - accura

In [6]:
a = model.evaluate(X_train,Y_train)

782/782 [==============================] - 5s 5ms/step - loss: 0.0030 - accuracy: 0.9996


In [7]:
a = model.evaluate(X_test,Y_test)

782/782 [==============================] - 4s 5ms/step - loss: 0.7864 - accuracy: 0.8117


In [9]:
print('loss:', a[0], 'acc:', a[1])

loss: 0.7864482998847961 acc: 0.8117200136184692
